# Tinh Chỉnh Mô Hình Open AI

Sổ tay này dựa trên hướng dẫn hiện tại được cung cấp trong tài liệu [Tinh Chỉnh](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) từ Open AI.

Tinh chỉnh cải thiện hiệu suất của các mô hình nền tảng cho ứng dụng của bạn bằng cách huấn luyện lại với dữ liệu bổ sung và ngữ cảnh liên quan đến trường hợp sử dụng hoặc kịch bản cụ thể đó. Lưu ý rằng các kỹ thuật thiết kế prompt như _học ít mẫu_ và _tăng cường tạo bằng truy xuất_ cho phép bạn nâng cao prompt mặc định với dữ liệu liên quan để cải thiện chất lượng. Tuy nhiên, các phương pháp này bị giới hạn bởi kích thước cửa sổ token tối đa của mô hình nền tảng mục tiêu.

Với tinh chỉnh, chúng ta thực sự đang huấn luyện lại chính mô hình với dữ liệu cần thiết (cho phép chúng ta sử dụng nhiều ví dụ hơn so với những gì có thể chứa trong cửa sổ token tối đa) - và triển khai một phiên bản _tùy chỉnh_ của mô hình mà không còn cần phải cung cấp ví dụ tại thời điểm suy luận. Điều này không chỉ cải thiện hiệu quả thiết kế prompt của chúng ta (chúng ta có nhiều linh hoạt hơn trong việc sử dụng cửa sổ token cho các mục đích khác) mà còn có thể cải thiện chi phí (bằng cách giảm số lượng token cần gửi đến mô hình khi suy luận).

Tinh chỉnh có 4 bước:
1. Chuẩn bị dữ liệu huấn luyện và tải lên.
1. Chạy công việc huấn luyện để có được mô hình đã được tinh chỉnh.
1. Đánh giá mô hình đã tinh chỉnh và lặp lại để nâng cao chất lượng.
1. Triển khai mô hình đã tinh chỉnh để suy luận khi hài lòng.

Lưu ý rằng không phải tất cả các mô hình nền tảng đều hỗ trợ tinh chỉnh - [kiểm tra tài liệu OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) để biết thông tin mới nhất. Bạn cũng có thể tinh chỉnh một mô hình đã được tinh chỉnh trước đó. Trong hướng dẫn này, chúng ta sẽ sử dụng `gpt-35-turbo` làm mô hình nền tảng mục tiêu để tinh chỉnh.

---


### Bước 1.1: Chuẩn bị Bộ Dữ liệu của Bạn

Hãy xây dựng một chatbot giúp bạn hiểu bảng tuần hoàn các nguyên tố bằng cách trả lời các câu hỏi về một nguyên tố dưới dạng một bài thơ limerick. Trong _hướng dẫn_ đơn giản này, chúng ta sẽ chỉ tạo một bộ dữ liệu để huấn luyện mô hình với một vài ví dụ mẫu về các phản hồi thể hiện định dạng dữ liệu mong đợi. Trong trường hợp sử dụng thực tế, bạn sẽ cần tạo một bộ dữ liệu với nhiều ví dụ hơn nhiều. Bạn cũng có thể sử dụng một bộ dữ liệu mở (cho lĩnh vực ứng dụng của bạn) nếu có, và định dạng lại để sử dụng trong việc tinh chỉnh.

Vì chúng ta tập trung vào `gpt-35-turbo` và tìm kiếm phản hồi một lượt (hoàn thành chat) nên chúng ta có thể tạo các ví dụ sử dụng [định dạng được đề xuất này](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) phản ánh các yêu cầu hoàn thành chat của OpenAI. Nếu bạn mong đợi nội dung hội thoại nhiều lượt, bạn sẽ sử dụng [định dạng ví dụ nhiều lượt](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) bao gồm tham số `weight` để báo hiệu những tin nhắn nào nên được sử dụng (hoặc không) trong quá trình tinh chỉnh.

Chúng ta sẽ sử dụng định dạng một lượt đơn giản hơn cho hướng dẫn ở đây. Dữ liệu ở định dạng [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) với 1 bản ghi trên mỗi dòng, mỗi bản ghi được biểu diễn dưới dạng một đối tượng JSON. Đoạn mã dưới đây hiển thị 2 bản ghi làm ví dụ - xem [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) để xem bộ mẫu đầy đủ (10 ví dụ) mà chúng ta sẽ sử dụng cho hướng dẫn tinh chỉnh. **Lưu ý:** Mỗi bản ghi _phải_ được định nghĩa trên một dòng duy nhất (không chia nhỏ trên nhiều dòng như trong file JSON định dạng thông thường)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Trong trường hợp sử dụng thực tế, bạn sẽ cần một bộ ví dụ lớn hơn nhiều để có kết quả tốt - sự đánh đổi sẽ là giữa chất lượng phản hồi và thời gian/chi phí cho việc tinh chỉnh. Chúng tôi đang sử dụng một bộ nhỏ để có thể hoàn thành việc tinh chỉnh nhanh chóng nhằm minh họa quy trình. Xem [ví dụ trong OpenAI Cookbook này](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) cho một hướng dẫn tinh chỉnh phức tạp hơn.


---

### Bước 1.2 Tải lên Bộ dữ liệu của bạn

Tải dữ liệu lên bằng cách sử dụng Files API [như được mô tả ở đây](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Lưu ý rằng để chạy được mã này, bạn phải đã thực hiện các bước sau trước:
 - Cài đặt gói Python `openai` (đảm bảo bạn sử dụng phiên bản >=0.28.0 để có các tính năng mới nhất)
 - Đặt biến môi trường `OPENAI_API_KEY` thành khóa API OpenAI của bạn
Để tìm hiểu thêm, xem [Hướng dẫn cài đặt](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) được cung cấp cho khóa học.

Bây giờ, chạy mã để tạo một tệp để tải lên từ tệp JSONL cục bộ của bạn.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Bước 2.1: Tạo công việc Fine-tuning với SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Bước 2.2: Kiểm tra trạng thái của công việc

Dưới đây là một số việc bạn có thể làm với API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Liệt kê n công việc fine-tuning cuối cùng
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Lấy chi tiết của một công việc fine-tuning cụ thể
- `client.fine_tuning.jobs.cancel(<job_id>)` - Hủy một công việc fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Liệt kê tối đa n sự kiện từ công việc
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Bước đầu tiên của quy trình là _xác thực file đào tạo_ để đảm bảo dữ liệu đúng định dạng.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Bước 2.3: Theo dõi sự kiện để giám sát tiến trình


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Bước 2.4: Xem trạng thái trong Bảng điều khiển OpenAI


Bạn cũng có thể xem trạng thái bằng cách truy cập trang web OpenAI và khám phá phần _Fine-tuning_ của nền tảng. Điều này sẽ hiển thị cho bạn trạng thái của công việc hiện tại, đồng thời cho phép bạn theo dõi lịch sử các lần thực thi công việc trước đó. Trong ảnh chụp màn hình này, bạn có thể thấy rằng lần thực thi trước đã thất bại, và lần chạy thứ hai đã thành công. Để làm rõ, điều này xảy ra khi lần chạy đầu tiên sử dụng một tệp JSON với các bản ghi định dạng sai - sau khi được sửa, lần chạy thứ hai đã hoàn thành thành công và làm cho mô hình có thể sử dụng được.

![Fine-tuning job status](../../../../../translated_images/vi/fine-tuned-model-status.563271727bf7bfba.png)


Bạn cũng có thể xem các thông báo trạng thái và số liệu bằng cách cuộn xuống thêm trong bảng điều khiển trực quan như hình dưới đây:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/vi/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/vi/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### Bước 3.1: Lấy ID & Kiểm tra Mô hình Tinh chỉnh trong Mã nguồn


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Bước 3.2: Tải & Kiểm Tra Mô Hình Đã Tinh Chỉnh Trong Playground

Bây giờ bạn có thể kiểm tra mô hình đã tinh chỉnh theo hai cách. Đầu tiên, bạn có thể truy cập Playground và sử dụng menu thả xuống Models để chọn mô hình đã tinh chỉnh mới của bạn từ các tùy chọn được liệt kê. Lựa chọn khác là sử dụng tùy chọn "Playground" hiển thị trong bảng Fine-tuning (xem ảnh chụp màn hình phía trên) để khởi chạy chế độ xem _so sánh_ sau đây, hiển thị các phiên bản mô hình nền tảng và mô hình đã tinh chỉnh cạnh nhau để đánh giá nhanh.

![Fine-tuning job status](../../../../../translated_images/vi/fine-tuned-playground-compare.56e06f0ad8922016.png)

Chỉ cần điền vào ngữ cảnh hệ thống được sử dụng trong dữ liệu huấn luyện của bạn và cung cấp câu hỏi kiểm tra. Bạn sẽ nhận thấy rằng cả hai bên đều được cập nhật với cùng một ngữ cảnh và câu hỏi giống hệt nhau. Chạy so sánh và bạn sẽ thấy sự khác biệt trong kết quả đầu ra giữa chúng. _Lưu ý cách mô hình đã tinh chỉnh trả lời theo định dạng bạn đã cung cấp trong các ví dụ của mình trong khi mô hình nền tảng chỉ đơn giản theo lời nhắc hệ thống_.

![Fine-tuning job status](../../../../../translated_images/vi/fine-tuned-playground-launch.5a26495c983c6350.png)

Bạn sẽ nhận thấy rằng so sánh cũng cung cấp số lượng token cho mỗi mô hình, và thời gian thực hiện suy luận. **Ví dụ cụ thể này là một ví dụ đơn giản nhằm minh họa quy trình nhưng không phản ánh dữ liệu hoặc kịch bản thực tế**. Bạn có thể nhận thấy rằng cả hai mẫu đều hiển thị cùng số lượng token (ngữ cảnh hệ thống và lời nhắc người dùng giống hệt nhau) với mô hình đã tinh chỉnh mất nhiều thời gian hơn để suy luận (mô hình tùy chỉnh).

Trong các kịch bản thực tế, bạn sẽ không sử dụng một ví dụ đơn giản như thế này, mà sẽ tinh chỉnh dựa trên dữ liệu thực (ví dụ: danh mục sản phẩm cho dịch vụ khách hàng) nơi chất lượng phản hồi sẽ rõ ràng hơn nhiều. Trong _ngữ cảnh đó_, để có được chất lượng phản hồi tương đương với mô hình nền tảng sẽ đòi hỏi kỹ thuật tạo lời nhắc tùy chỉnh nhiều hơn, điều này sẽ làm tăng việc sử dụng token và có thể tăng thời gian xử lý suy luận liên quan. _Để thử điều này, hãy xem các ví dụ tinh chỉnh trong OpenAI Cookbook để bắt đầu._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Tuyên bố từ chối trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ gốc của nó nên được coi là nguồn chính xác và đáng tin cậy. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp do con người thực hiện. Chúng tôi không chịu trách nhiệm về bất kỳ sự hiểu lầm hoặc giải thích sai nào phát sinh từ việc sử dụng bản dịch này.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
